In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [8]:
from sagemaker import model_uris, image_uris, script_uris
import tempfile
import os
import subprocess


model_uri = model_uris.retrieve(
    model_id=os.environ["MODEL_ID"],
    model_version=os.environ["MODEL_VERSION"],
    model_scope="inference",
)

temp_dir = tempfile.mkdtemp()
model_dir = os.path.join(temp_dir, "model")
os.makedirs(model_dir, exist_ok=True)

subprocess.run(
    ["aws", "s3", "cp", "--recursive", model_uri, model_dir, "--quiet"],
    check=True,
)
subprocess.run(
    [
        "tar",
        "-czf",
        os.path.join(temp_dir, "model.tar.gz"),
        "-C",
        model_dir,
        ".",
    ],
    check=True,
)
subprocess.run(
    [
        "aws",
        "s3",
        "cp",
        os.path.join(temp_dir, "model.tar.gz"),
        os.environ["S3_PRETRAINED_MODEL_URI"],
        "--quiet",
    ],
    check=True,
)

deploy_image_uri = image_uris.retrieve(
    region="eu-north-1",
    framework=None,
    image_scope="inference",
    model_id=os.environ["MODEL_ID"],
    model_version=os.environ["MODEL_VERSION"],
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
)

deploy_source_uri = script_uris.retrieve(
    model_id=os.environ["MODEL_ID"], model_version=os.environ["MODEL_VERSION"], script_scope="inference"
)

In [9]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=os.environ["S3_PRETRAINED_MODEL_URI"],
        entry_point="inference.py",
        role=os.environ["SM_EXEC_ROLE"],
        predictor_cls=Predictor,
        name="tf-ic-birds-pretrained",
        code_location=os.environ["S3_PRETRAINED_OUTPUT_URI"],
    )

model.deploy(
    initial_instance_count=int(os.environ["INFERENCE_INSTANCE_COUNT"]),
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
    endpoint_name="tf-ic-birds-pretrained-ep",
    endpoint_logging=True,
)

[03/03/25 23:35:19] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=595517;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=807505;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/botocore/credentials.py#1352\1352]8;;\

                    INFO     Repacking model artifact                                                  ]8;id=197853;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=775225;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/model.py#819\819]8;;\
                             (s3://tf-ic-birds/pretrained/model/model.tar.gz), script artifact                     
                             (s3://jumpstart-cache-prod-eu-north-1/source-directory-tarballs/tensorflo             
                             w/inference/ic/v2.0.3/sourcedir.tar.gz), and dependencies ([]) into                   
                             single tar.gz file located at                                                         
                             s3://tf-ic-birds/pretrained/output/tf-ic-birds-pretrained/model.tar.gz.               
                             This may take some time depending on model size...                                    

[03/03/25 23:36:01] INFO     Creating model with name: tf-ic-birds-pretrained                       ]8;id=625239;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=543723;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/03/25 23:36:03] WARNING  Using already existing model: tf-ic-birds-pretrained                   ]8;id=344371;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=861534;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name tf-ic-birds-pretrained-ep           ]8;id=657321;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=782582;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name tf-ic-birds-pretrained-ep                  ]8;id=491170;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=566535;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\

[03/03/25 23:36:04] WARNING  Failed to enable live logging: An error occurred                       ]8;id=404757;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=105059;file:///home/mahmood/tf-image-classification/.venv/lib/python3.12/site-packages/sagemaker/session.py#8805\8805]8;;\
                             (AccessDeniedException) when calling the FilterLogEvents operation:                   
                             User: arn:aws:iam::482497089777:user/mahmood is not authorized to                     
                             perform: logs:FilterLogEvents on resource:                                            
                             arn:aws:logs:eu-north-1:482497089777:log-group:/aws/sagemaker/Endpoint                
                             s/tf-ic-birds-pretrained-ep:log-stream: because no identity-based                     
                             policy allows the logs:FilterLogEvents action. Fallback to default                    
                             logging...                                                                            

-----------!